In [ ]:
#Modelo para predecir tendencia mensual (clasificador ML: Sube o baja)
#Necesitamos muchos mas datos, de varias acciones: entrenamos el modelo pej en los años 2020-2023 y testeamos en el 24
import pickle
import pandas as pd

with open("ibex_cache.pkl", "rb") as f:
    data = pickle.load(f)

price_data = data["price_data"]
info_meta = data["info_meta"]

# Coger un ticker cualquiera
sample_ticker = list(price_data.keys())[0]
sample = price_data[sample_ticker]

print("Ticker de ejemplo:", sample_ticker)
print("Tipo interno:", type(sample))
print("Contenido ejemplo:")
print(sample)


In [ ]:
import pandas as pd

panel = pd.concat(price_data, axis=1)  # columnas MultiIndex (ticker, campo)
panel.index = pd.to_datetime(panel.index)
panel = panel.sort_index()

print(panel.columns.levels)
print(panel.head())


In [ ]:
ticker = "ACS.MC"
df = price_data[ticker].copy()

# Asegurar formato
df.index = pd.to_datetime(df.index)
df = df.sort_index()

print(df.head())


In [ ]:
import numpy as np
import pandas as pd

price_col = "Adj Close"

quant = pd.DataFrame(index=df.index)
quant["price"] = df[price_col]

quant["ret_simple"] = quant["price"].pct_change()
quant["ret_log"] = np.log(quant["price"] / quant["price"].shift(1))

quant = quant.dropna()

print(quant.head())
print(quant["ret_simple"].describe())


In [ ]:
quant["sma20"] = quant["price"].rolling(20).mean()
quant["sma50"] = quant["price"].rolling(50).mean()


In [ ]:
quant["signal"] = (quant["sma20"] > quant["sma50"]).astype(int)


In [ ]:
quant["strategy_ret"] = quant["signal"].shift(1) * quant["ret_simple"]


In [ ]:
quant["equity_curve"] = (1 + quant["strategy_ret"]).cumprod()
quant["buy_hold"] = (1 + quant["ret_simple"]).cumprod()


In [ ]:
sharpe = quant["strategy_ret"].mean() / quant["strategy_ret"].std() * (252**0.5)
print("Sharpe:", sharpe)


In [ ]:
roll_max = quant["equity_curve"].cummax()
quant["drawdown"] = quant["equity_curve"] / roll_max - 1
print("Max DD:", quant["drawdown"].min())


In [ ]:
quant["signal"].value_counts()

In [ ]:
quant["equity_curve"].tail(1)

In [ ]:
import numpy as np

buy_hold_ret = quant["ret_simple"]
quant["buy_hold_equity"] = (1 + buy_hold_ret).cumprod()

print("Buy & hold final:", quant["buy_hold_equity"].iloc[-1])

sh_bh = buy_hold_ret.mean() / buy_hold_ret.std() * np.sqrt(252)
print("Sharpe buy&hold:", sh_bh)

roll_max_bh = quant["buy_hold_equity"].cummax()
dd_bh = quant["buy_hold_equity"] / roll_max_bh - 1
print("Max DD buy&hold:", dd_bh.min())


In [ ]:
cost = 0.001  # 0.1%

quant["trade"] = quant["signal"].diff().abs()  # 1 cuando hay cambio de posición
quant["strategy_ret_tc"] = quant["strategy_ret"] - quant["trade"] * cost
quant["equity_curve_tc"] = (1 + quant["strategy_ret_tc"]).cumprod()

print("Equity con TC:", quant["equity_curve_tc"].iloc[-1])


In [ ]:
import numpy as np
import pandas as pd

results = []

for short in [5,10,15,20,30]:
    for long in [20,40,60,80,100]:
        if short >= long:
            continue
        
        tmp = quant.copy()
        tmp["sma_s"] = tmp["price"].rolling(short).mean()
        tmp["sma_l"] = tmp["price"].rolling(long).mean()
        tmp["signal"] = (tmp["sma_s"] > tmp["sma_l"]).astype(int)

        tmp["strategy_ret"] = tmp["signal"].shift(1) * tmp["ret_simple"]
        tmp = tmp.dropna()

        sharpe = tmp["strategy_ret"].mean() / tmp["strategy_ret"].std() * np.sqrt(252)
        final = (1 + tmp["strategy_ret"]).cumprod().iloc[-1]

        results.append((short, long, sharpe, final))

df_res = pd.DataFrame(results, columns=["short", "long", "sharpe", "final"])
print(df_res.sort_values("sharpe", ascending=False).head(10))


In [ ]:
import pandas as pd

panel = pd.concat(price_data, axis=1)  # columnas MultiIndex: (ticker, campo)
panel.index = pd.to_datetime(panel.index)
panel = panel.sort_index()

# Nos quedamos solo con Adj Close para cada ticker
adj_close = panel.xs('Adj Close', level=1, axis=1)

print(adj_close.head())
print(adj_close.columns)


In [ ]:
# Precio de cierre de cada mes (último día hábil del mes)
monthly_prices = adj_close.resample("M").last()

# Retornos mensuales simples
monthly_ret = monthly_prices.pct_change()

print(monthly_prices.tail())
print(monthly_ret.tail())


In [ ]:
momentum_6m = monthly_prices.pct_change(6)

print(momentum_6m.tail())


In [ ]:
# Rank descendente: 1 = mejor momentum
ranks = momentum_6m.rank(axis=1, ascending=False)

print(ranks.tail())


In [ ]:
top_n = 5
longs = ranks <= top_n   # DataFrame booleano


In [ ]:
# Shift para evitar look-ahead bias: usamos la señal de mes anterior
signal_long = longs.shift(1)

# Retorno de la estrategia = media de retornos de los activos en cartera
strategy_ret = (signal_long * monthly_ret).sum(axis=1) / signal_long.sum(axis=1)

# Si algún mes no hay activos seleccionados (por NaNs), mejor poner 0
strategy_ret = strategy_ret.fillna(0.0)

equity_cs = (1 + strategy_ret).cumprod()

print(strategy_ret.describe())
print("Equity final cross-sectional:", equity_cs.iloc[-1])


In [ ]:
# Cartera equiponderada: todos los activos con igual peso
ew_ret = monthly_ret.mean(axis=1)
ew_equity = (1 + ew_ret).cumprod()

print("Equity final EW:", ew_equity.iloc[-1])
